In [1]:
import pyarrow.parquet as pq
import pandas as pd
import dask_geopandas as dgp
from shapely import wkb
import matplotlib.pyplot as plt
import geopandas as gpd

In [15]:
base_path = '~/src/project/cmr-bigstac-prototype/bigstac/scripts_explore/partitioning'

input_file = '3mil_no_global_bounds.parquet'

for i in range(5):
  input_file = f"{base_path}/part.{i}.parquet"
  ddf = dgp.read_parquet(input_file)
  print(ddf.head(3)['geometry'])

hilbert_distance
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
Name: geometry, dtype: geometry
hilbert_distance
967256469    POLYGON ((-146.01353 -33.11766, -171.93548 -36...
967256469    POLYGON ((-146.01353 -33.11766, -171.93548 -36...
967256469    POLYGON ((-146.01353 -33.11766, -171.93548 -36...
Name: geometry, dtype: geometry
hilbert_distance
1893404616    POLYGON ((-9.46232 33.08863, 0 33.93548, 0 43....
1893404616    POLYGON ((-9.46232 33.08863, 0 33.93548, 0 43....
1893404616    POLYGON ((-9.46232 33.08863, 0 33.93548, 0 43....
Name: geometry, dtype: geometry
hilbert_distance
2372220553    POLYGON ((39.10132 22.51865, 39.35742 23.51015...
2372220553    POLYGON ((39.10132 22.51865, 39.35742 23.51015...
2372220816    POLYGON ((38.67927 22.97668, 38.71723 22.79244...
Name: geometry, dtype: geometry
hilbert_distance
3315025670    POLYGON ((166.36975 -44.37806

In [16]:
i = 0
input_file = f"{base_path}/part.{i}.parquet"
ddf = dgp.read_parquet(input_file)
print(ddf.head()['geometry'])
print(ddf.tail()['geometry'])

hilbert_distance
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
877    POLYGON ((-180 -89.9, -180 -90, -179.9 -90, -1...
Name: geometry, dtype: geometry
hilbert_distance
967256199    POLYGON ((-146.52577 -33.24356, -171.923 -36.8...
967256199    POLYGON ((-136.29118 -49.92588, -141.86511 -42...
967256199    POLYGON ((-146.52577 -33.24356, -171.923 -36.8...
967256199    POLYGON ((-146.52577 -33.24356, -171.923 -36.8...
967256199    POLYGON ((-146.52577 -33.24356, -171.923 -36.8...
Name: geometry, dtype: geometry


In [17]:
base_path = '~/src/project/cmr-bigstac-prototype/bigstac/scripts_explore/partitioning'
i = 0
input_file = f"{base_path}/part.{i}.parquet"
print(input_file)
parquet_file = pq.ParquetFile(input_file)

# Loop through each row group
for row_group_index in range(parquet_file.num_row_groups):
    row_group = parquet_file.read_row_group(row_group_index)
    df = row_group.to_pandas()
    i = 0

    last_start = None
    last_geo = None
    for index, row in df.iterrows():
        time_start = row['StartTime']
        time_stop = row['EndTime']
        
        #if last_start is not None and last_start < time_stop:
        #  print(f"out of order {i}: {time_start} - {time_stop}")
        #last_start = time_stop
        
        # Convert the WKB geometry to a Shapely geometry object
        geometry = wkb.loads(row['geometry'], hex=True)
        #print(geometry)

        if last_geo is not None:
          if last_geo != geometry:
            i = i + 1
            print(f"geom changed {i}: {last_geo.centoid.x < geometry.centroid.x}")
        last_geo = geometry

        #print(geometry.bounds)
        #print(geometry.geom_type)
        #print(geometry.area)
        #print(geometry.length)
        #bbox = row['bbox']
        #print (f"{index}: {time_start} - {geometry}")
        #print(row['StartTime', 'EndTime', 'geometry', 'bbox']) 
        if  i >= 15:
          break

    print(f"Finished processing row group {row_group_index}")

print("Finished processing all row groups")


~/src/project/cmr-bigstac-prototype/bigstac/scripts_explore/partitioning/part.0.parquet


AttributeError: 'Polygon' object has no attribute 'centoid'